In [1]:
import os
import numpy as np
import sklearn
# from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras.utils import Sequence
# from keras.utils import to_categorical
import sys


sys.path.append('..')

from preprocess.spectrogram import plot_mel_spect
# from preprocess.wav_helper import trim_audio_to_np_float

from preprocess.preprocess import preprocess, make_spects

from model.load import load_model


2025-11-03 13:54:28.866009: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-03 13:54:29.855169: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-03 13:54:35.607756: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-03 13:54:38.946640: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:

In [5]:
tf.version

<module 'tensorflow._api.v2.version' from '/home/x3nom/.local/lib/python3.12/site-packages/tensorflow/_api/v2/version/__init__.py'>

In [7]:
keras.version()

'3.11.3'

In [2]:
LABEL_NAMES = ['3S', 'BC', 'BD', 'BE', 'BhBl', 'BlBh', 'XlB', 'XsB']

In [3]:
model = load_model('../../model.keras')

In [4]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 192000)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mel_spectrogram_2               │ (None, 192, 376)       │             0 │
│ (MelSpectrogram)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_2 (Lambda)               │ (None, 192, 376, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ densenet121 (Functional)        │ (None, 1024)           │     7,037,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │       262,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 11)             │           715 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,783,907 (83.10 MB)

 Trainable params: 7,233,419 (27.59 MB)

 Non-trainable params: 83,648 (326.75 KB)

 Optimizer params: 14,466,840 (55.19 MB)

In [ ]:
FP = "../../.tstdata/F002413.wav"

samples = preprocess(FP)

for sample in samples:
    # X = np.zeros((1, *spect.shape, 1), dtype=np.float32)
    # X[0, :, :, 0] = spect

    audio_tensor = tf.convert_to_tensor(
        np.asarray(sample).reshape(1, -1), dtype=tf.float32
    )

    prediction = model.predict(
        audio_tensor
    )


    pred_percent = dict(zip(LABEL_NAMES, map(lambda x: f"{round(float(x), 2) * 100}%", prediction.flatten())))
    percent_str = ' | '.join([ f"{k}:{pred_percent[k]}" for k in pred_percent.keys() ])

    print(percent_str)
    # plot_mel_spect(, title=percent_str)